In [1]:
import decouple

In [2]:
config = decouple.AutoConfig(' ')

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [4]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [5]:
ALPHA_VANTAGE_API_KEY

'4DA3RF1H7L46HH9X'

In [6]:
import requests

In [7]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "function": "TIME_SERIES_INTRADAY",
    "ticker": "AAPL",
    "interval": "5min"
}

In [8]:
url = "https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}".format(
    **params
)

'https://www.alphavantage.co/query?apikey=4DA3RF1H7L46HH9X&function=TIME_SERIES_INTRADAY&ticker=AAPL&interval=5min'

In [9]:
r = requests.get(url)

In [10]:
data = r.json()

In [11]:
# data

In [12]:
data.keys()

dict_keys(['Meta Data', 'Time Series (5min)'])

In [13]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]

In [14]:
dataset_key

'Time Series (5min)'

In [15]:
results = data[dataset_key]

In [16]:
# results[0]

In [17]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2025-01-15 19:55:00'

In [18]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '238.6000',
 '2. high': '238.6000',
 '3. low': '238.4500',
 '4. close': '238.5000',
 '5. volume': '2099'}

In [19]:
from datetime import datetime
import pytz
from decimal import Decimal

def transform_alphavantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    timestamp
    return {
        "volume": int(result["5. volume"]),
        "open": Decimal(result["1. open"]),
        "close": Decimal(result["4. close"]),
        "high": Decimal(result["2. high"]),
        "low": Decimal(result["3. low"]),
        "timestamp": timestamp,  
    }

In [20]:
transform_alphavantage_result(timestamp_str, result_val)

{'volume': 2099,
 'open': Decimal('238.6000'),
 'close': Decimal('238.5000'),
 'high': Decimal('238.6000'),
 'low': Decimal('238.4500'),
 'timestamp': datetime.datetime(2025, 1, 16, 0, 55, tzinfo=<UTC>)}

In [21]:
for timestamp in results.keys():
    new_data = transform_alphavantage_result(timestamp, results.get(timestamp))
    # print(new_data)

In [44]:
import pytz

from dataclasses import dataclass
from datetime import datetime
from typing import Literal
from urllib.parse import urlencode

def transform_alphavantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    timestamp
    return {
        "volume": int(result["5. volume"]),
        "open": Decimal(result["1. open"]),
        "close": Decimal(result["4. close"]),
        "high": Decimal(result["2. high"]),
        "low": Decimal(result["3. low"]),
        "timestamp": timestamp,  
    }
    
@dataclass
class AlphaVantageAPIClient:
    ticker: str = "AAPL"
    function: str = "TIME_SERIES_INTRADAY"
    interval: str = "5min"
    api_key: str = ""
    month: str = "2024-12"

    def get_api_keys(self):
        return self.api_key or ALPHA_VANTAGE_API_KEY
        
    def get_headers(self):
        return {
            "Authorization": f"Bearer {self.get_api_keys()}"
        }

    def get_params(self):
        return {
            "apikey": self.get_api_keys(),
            "function": self.function,
            "symbol": self.ticker,
            "interval": self.interval,
            "month": self.month
        }

    def generate_url(self):
        url = "https://www.alphavantage.co/query"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # not for 200/201
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        # print(data)
        dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
        results = data[dataset_key]
        return [transform_alphavantage_result(timestamp, results.get(timestamp)) for timestamp in results.keys()]

In [45]:
alphavantage_client = AlphaVantageAPIClient()
stock_data = alphavantage_client.get_stock_data()
# stock_data = alphavantage_client.generate_url()
stock_data

[{'volume': 925,
  'open': Decimal('250.4200'),
  'close': Decimal('250.4500'),
  'high': Decimal('250.4700'),
  'low': Decimal('250.4100'),
  'timestamp': datetime.datetime(2025, 1, 1, 0, 55, tzinfo=<UTC>)},
 {'volume': 304,
  'open': Decimal('250.4450'),
  'close': Decimal('250.4500'),
  'high': Decimal('250.4800'),
  'low': Decimal('250.4100'),
  'timestamp': datetime.datetime(2025, 1, 1, 0, 50, tzinfo=<UTC>)},
 {'volume': 271,
  'open': Decimal('250.4450'),
  'close': Decimal('250.4450'),
  'high': Decimal('250.4800'),
  'low': Decimal('250.4300'),
  'timestamp': datetime.datetime(2025, 1, 1, 0, 45, tzinfo=<UTC>)},
 {'volume': 700,
  'open': Decimal('250.4200'),
  'close': Decimal('250.4500'),
  'high': Decimal('250.4500'),
  'low': Decimal('250.4100'),
  'timestamp': datetime.datetime(2025, 1, 1, 0, 40, tzinfo=<UTC>)},
 {'volume': 961,
  'open': Decimal('250.4150'),
  'close': Decimal('250.4150'),
  'high': Decimal('250.4200'),
  'low': Decimal('250.4100'),
  'timestamp': datetime